In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
#converting categorical to numerical value
dataset=pd.get_dummies(dataset,dtype=int, drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
indep.shape

(400, 3)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Define the parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],  # Smoothing parameter
    'binarize': [0.0, 0.5, 1.0]  # Threshold for binarizing (if not already binary)
}

# Set up the GridSearchCV object
grid = GridSearchCV(BernoulliNB(), param_grid, 
                    refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fitting the model for grid search
grid.fit(X_train, y_train)

# To see the best parameters and the best estimator
print("Best Parameters:", grid.best_params_)
print("Best Estimator:", grid.best_estimator_)




Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best Parameters: {'alpha': 0.1, 'binarize': 0.0}
Best Estimator: BernoulliNB(alpha=0.1)


In [13]:
# Store cross-validation results
cv_results = grid.cv_results_
# Uncomment to print all cross-validation results
# print(cv_results)

# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)

# Using the final best fit model to test X_test
grid_predictions = grid.predict(X_test)

In [14]:
# Compute and print the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,grid_predictions )  

In [15]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

C:\Anaconda\envs\aiml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda\envs\aiml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda\envs\aiml\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.63      1.00      0.78        85
           1       0.00      0.00      0.00        49

    accuracy                           0.63       134
   macro avg       0.32      0.50      0.39       134
weighted avg       0.40      0.63      0.49       134



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.49039615846338536)

In [18]:
print(cm)

[[85  0]
 [49  0]]


In [19]:
grid.predict([[40,300,1,]])

C:\Anaconda\envs\aiml\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but BernoulliNB was fitted with feature names
  warnings.warn(


array([0])

In [20]:
table=pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_binarize,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013731,0.003619,0.009352,0.005623,0.1,0.0,"{'alpha': 0.1, 'binarize': 0.0}",0.509779,0.5253,0.50141,0.50141,0.50141,0.507862,0.009302,1
1,0.011546,0.004766,0.011561,0.005127,0.1,0.5,"{'alpha': 0.1, 'binarize': 0.5}",0.509779,0.5253,0.50141,0.50141,0.50141,0.507862,0.009302,1
2,0.005969,0.003247,0.014223,0.001909,0.1,1.0,"{'alpha': 0.1, 'binarize': 1.0}",0.509779,0.5253,0.50141,0.50141,0.50141,0.507862,0.009302,1
3,0.016616,0.010213,0.018215,0.006808,0.5,0.0,"{'alpha': 0.5, 'binarize': 0.0}",0.509779,0.5253,0.50141,0.50141,0.50141,0.507862,0.009302,1
4,0.006604,0.003850,0.014775,0.003741,0.5,0.5,"{'alpha': 0.5, 'binarize': 0.5}",0.509779,0.5253,0.50141,0.50141,0.50141,0.507862,0.009302,1
5,0.011207,0.001816,0.011059,0.001495,0.5,1.0,"{'alpha': 0.5, 'binarize': 1.0}",0.509779,0.5253,0.50141,0.50141,0.50141,0.507862,0.009302,1
6,0.013160,0.001809,0.011730,0.002457,1.0,0.0,"{'alpha': 1.0, 'binarize': 0.0}",0.509779,0.5253,0.50141,0.50141,0.50141,0.507862,0.009302,1
7,0.011378,0.002920,0.012562,0.003827,1.0,0.5,"{'alpha': 1.0, 'binarize': 0.5}",0.509779,0.5253,0.50141,0.50141,0.50141,0.507862,0.009302,1
8,0.010277,0.003562,0.010335,0.001482,1.0,1.0,"{'alpha': 1.0, 'binarize': 1.0}",0.509779,0.5253,0.50141,0.50141,0.50141,0.507862,0.009302,1
9,0.010231,0.003874,0.018788,0.004010,1.5,0.0,"{'alpha': 1.5, 'binarize': 0.0}",0.509779,0.5253,0.50141,0.50141,0.50141,0.507862,0.009302,1
